<a href="https://colab.research.google.com/github/DiogoMondin/json-table-handling/blob/main/T%26D_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importações

In [49]:
!apt-get install openjdk-11-jdk -qq > /dev/null
!pip install -q pyspark

In [64]:
import requests
import json
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

# Download JSON

In [45]:
url = 'https://health.data.ny.gov/api/views/jxy9-yhdk/rows.json'
response = requests.get(url)
data = response.json()

# Investigação das colunas

In [52]:
columns_meta = data['meta']['view']['columns']
column_names = [col['name'] for col in columns_meta]

print(column_names)

['sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta', 'Year', 'First Name', 'County', 'Sex', 'Count']


In [57]:
rows = data['data']

data_dicts = [dict(zip(column_names, row)) for row in rows]

In [58]:
spark = SparkSession.builder \
    .appName("JSON Baby Names") \
    .getOrCreate()

# Pré-definição de schema (inicialmente em StringType)

In [77]:
schema = T.StructType([
    T.StructField("sid",            T.StringType(), True),
    T.StructField("id",             T.StringType(), True),
    T.StructField("position",       T.StringType(), True),
    T.StructField("created_at",     T.StringType(), True),
    T.StructField("created_meta",   T.StringType(), True),
    T.StructField("updated_at",     T.StringType(), True),
    T.StructField("updated_meta",   T.StringType(), True),
    T.StructField("meta",           T.StringType(), True),
    T.StructField("Year",           T.StringType(), True),
    T.StructField("First Name",     T.StringType(), True),
    T.StructField("County",         T.StringType(), True),
    T.StructField("Sex",            T.StringType(), True),
    T.StructField("Count",          T.StringType(), True),
])

baby_names_df = spark.createDataFrame(data_dicts, schema=schema)

# Formatação final das colunas e criação de temp view

In [80]:
baby_names_df_formated = (
    baby_names_df
    .withColumn("sid",            F.col("sid").cast("string"))
    .withColumn("id",             F.col("id").cast("string"))
    .withColumn("position",       F.col("position").cast("int"))
    .withColumn("created_at",     F.from_unixtime(F.col("created_at")))
    .withColumn("updated_at",   F.from_unixtime(F.col("updated_at")))
    .withColumn("meta",           F.col("meta").cast("string"))
    .withColumn("Year",           F.col("Year").cast("int"))
    .withColumn("First Name",     F.col("First Name").cast("string"))
    .withColumn("County",         F.col("County").cast("string"))
    .withColumn("Sex",            F.col("Sex").cast("string"))
    .withColumn("Count",          F.col("Count").cast("int"))

    .drop("created_meta") # NULL values
    .drop("updated_meta") # NULL values
)

baby_names_df_formated.createOrReplaceTempView("birth_data")

baby_names_df_formated.show()

+------------------+--------------------+--------+-------------------+-------------------+----+----+----------+------+---+-----+
|               sid|                  id|position|         created_at|         updated_at|meta|Year|First Name|County|Sex|Count|
+------------------+--------------------+--------+-------------------+-------------------+----+----+----------+------+---+-----+
|row-ddjv_cm93_6icc|00000000-0000-000...|       0|2025-04-25 19:03:18|2025-04-25 19:03:18| { }|2022|    OLIVIA|Albany|  F|   16|
|row-dbx8_dtbn-e5i4|00000000-0000-000...|       0|2025-04-25 19:03:18|2025-04-25 19:03:18| { }|2022|    AMELIA|Albany|  F|   15|
|row-7bm2-ibrt_zigs|00000000-0000-000...|       0|2025-04-25 19:03:18|2025-04-25 19:03:18| { }|2022|     AVERY|Albany|  F|   12|
|row-twbr~qzdf.jnfm|00000000-0000-000...|       0|2025-04-25 19:03:18|2025-04-25 19:03:18| { }|2022|      EMMA|Albany|  F|   11|
|row-hxpw_hv5d.a7xc|00000000-0000-000...|       0|2025-04-25 19:03:18|2025-04-25 19:03:18| { }|20